In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.8.0+cu126


In [ ]:
!pip install sympy

In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.9 MB/s eta 0:00:00


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

In [19]:
fa_patients_path = "/home/snu/Downloads/NIFD_Patients_FA_Histogram_Feature.npy"
Patients_FA_array = np.load(fa_patients_path, allow_pickle=True)

fa_controls_path = "/home/snu/Downloads/NIFD_Control_FA_Histogram_Feature.npy"
Controls_FA_array = np.load(fa_controls_path, allow_pickle=True)

print("Patients Shape:", Patients_FA_array.shape)
print("Controls Shape:", Controls_FA_array.shape)

X = np.vstack([Controls_FA_array, Patients_FA_array])
y = np.hstack([
    np.zeros(Controls_FA_array.shape[0], dtype=np.int64),  # 0 = Control
    np.ones(Patients_FA_array.shape[0], dtype=np.int64)    # 1 = Patient
])

np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]

Patients Shape: (98, 180)
Controls Shape: (48, 180)


In [20]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = (W >= alpha).astype(np.float32)
    return W

In [21]:
def asymmetrize_random(adj_matrix, seed=None):
    """
    Randomly orient each undirected edge from a symmetric adjacency matrix.
    """
    adj = np.array(adj_matrix, dtype=np.float32)
    n = adj.shape[0]
    asym = np.zeros((n, n), dtype=np.float32)
    rng = np.random.default_rng(seed)

    for i in range(n):
        for j in range(i + 1, n):
            if adj[i, j]:
                if rng.random() < 0.5:
                    asym[i, j] = adj[i, j]
                else:
                    asym[j, i] = adj[i, j]

    return asym

In [36]:
def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero(adj))).long()
    return node_feats, edge_index

In [48]:
features = X.astype(np.float32)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

W0 = create_adj(features, alpha=0.5)
# W_asym = asymmetrize_random(W0, seed=42)
node_feats, edge_index = load_data(W0, features)
data = Data(x=node_feats, edge_index=edge_index).to(device)
A = torch.from_numpy(W0).to(device)
print(data)

Data(x=[146, 180], edge_index=[2, 21256])


In [49]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, device, activ):
        super(GCNEncoder, self).__init__()
        self.device = device
        self.gcn1 = GCNConv(input_dim, hidden_dim)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.25)
        self.mlp = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn1(x, edge_index)
        x = self.dropout(x)
        x = self.batchnorm(x)
        logits = self.mlp(x)
        return logits

In [50]:
class AvgReadout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, seq, msk=None):
        if msk is None:
            return torch.mean(seq, 0)
        else:
            msk = torch.unsqueeze(msk, -1)
            return torch.sum(seq * msk, 0) / torch.sum(msk)

In [51]:
class Discriminator(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.f_k = nn.Bilinear(n_h, n_h, 1)
        nn.init.xavier_uniform_(self.f_k.weight.data)
        if self.f_k.bias is not None:
            self.f_k.bias.data.fill_(0.0)

    def forward(self, c, h_pl, h_mi):
        c_x = torch.unsqueeze(c, 0).expand_as(h_pl)
        sc_1 = torch.squeeze(self.f_k(h_pl, c_x), 1)
        sc_2 = torch.squeeze(self.f_k(h_mi, c_x), 1)
        logits = torch.cat((sc_1, sc_2), 0)
        return logits

In [52]:
class DGI(nn.Module):
    def __init__(self, n_in, n_h, dropout=0.25):
        super().__init__()
        self.gcn1 = GCNEncoder(n_in, n_h, device='cuda' if torch.cuda.is_available() else 'cpu', activ=nn.ELU())
        self.read = AvgReadout()
        self.sigm = nn.Sigmoid()
        self.disc = Discriminator(n_h)

    def forward(self, seq1, seq2, edge_index):
        # Create Data objects for the GCNEncoder
        data1 = Data(x=seq1, edge_index=edge_index)
        data2 = Data(x=seq2, edge_index=edge_index)

        h_1 = self.gcn1(data1)
        c = self.read(h_1)
        c = self.sigm(c)
        h_2 = self.gcn1(data2)
        logits = self.disc(c, h_1, h_2)
        return logits, h_1

In [61]:
class DGI_with_classifier(DGI):
    def __init__(self, n_in, n_h, n_classes=2, cut=0, dropout=0.25):
        super().__init__(n_in, n_h, dropout=dropout)
        self.classifier = nn.Linear(n_h, n_classes)
        self.cut = cut

    def get_embeddings(self, node_feats, edge_index):
        _, embeddings = self.forward(node_feats, node_feats, edge_index)
        return embeddings

    def cut_loss(self, A, S):
        S = F.softmax(S, dim=1)
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)
        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)
        mincut_loss = -(num / den)
        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(S.shape[1], device=A.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))
        return mincut_loss + ortho_loss

    def modularity_loss(self, A, S):
        C = F.softmax(S, dim=1)
        d = torch.sum(A, dim=1)
        m = torch.sum(A)
        B = A - torch.ger(d, d) / (2 * m)
        I_S = torch.eye(C.shape[1], device=A.device)
        k = torch.norm(I_S)
        n = S.shape[0]
        modularity_term = (-1 / (2 * m)) * torch.trace(torch.mm(torch.mm(C.t(), B), C))
        collapse_reg_term = (torch.sqrt(k) / n) * torch.norm(torch.sum(C, dim=0), p='fro') - 1
        return modularity_term + collapse_reg_term

    def Reg_loss(self, A, embeddings):
        logits = self.classifier(embeddings)
        if self.cut == 1:
            return self.cut_loss(A, logits)
        else:
            return self.modularity_loss(A, logits)

In [62]:
hidden_dim = 512
cut = 0
dropout = 0.3
model = DGI_with_classifier(features.shape[1], hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.00001)
bce_loss = nn.BCEWithLogitsLoss()

num_epochs = 8000
for epoch in range(num_epochs + 1):
    model.train()
    optimizer.zero_grad()

    perm = torch.randperm(node_feats.size(0))
    corrupt_features = node_feats[perm]

    logits, embeddings = model(node_feats.to(device), corrupt_features.to(device), edge_index.to(device))

    lbl = torch.cat([
        torch.ones(node_feats.size(0)),
        torch.zeros(node_feats.size(0))
    ]).to(device)

    dgi_loss = bce_loss(logits.squeeze(), lbl)
    reg_loss = model.Reg_loss(A, embeddings)
    loss = dgi_loss + 0.5 * reg_loss

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | DGI Loss: {dgi_loss.item():.4f} | Reg Loss: {reg_loss.item():.4f} | Total: {loss.item():.4f}")

    loss.backward()
    optimizer.step()

Epoch 0 | DGI Loss: 0.7116 | Reg Loss: -0.2839 | Total: 0.5697
Epoch 500 | DGI Loss: 0.7311 | Reg Loss: -0.2841 | Total: 0.5891
Epoch 1000 | DGI Loss: 0.7105 | Reg Loss: -0.2841 | Total: 0.5684
Epoch 1500 | DGI Loss: 0.6968 | Reg Loss: -0.2842 | Total: 0.5547
Epoch 2000 | DGI Loss: 0.6961 | Reg Loss: -0.2845 | Total: 0.5539
Epoch 2500 | DGI Loss: 0.6935 | Reg Loss: -0.2846 | Total: 0.5512
Epoch 3000 | DGI Loss: 0.7015 | Reg Loss: -0.2844 | Total: 0.5593
Epoch 3500 | DGI Loss: 0.6933 | Reg Loss: -0.2844 | Total: 0.5511
Epoch 4000 | DGI Loss: 0.6938 | Reg Loss: -0.2845 | Total: 0.5516
Epoch 4500 | DGI Loss: 0.6935 | Reg Loss: -0.2846 | Total: 0.5512
Epoch 5000 | DGI Loss: 0.6933 | Reg Loss: -0.2846 | Total: 0.5510
Epoch 5500 | DGI Loss: 0.6932 | Reg Loss: -0.2845 | Total: 0.5510
Epoch 6000 | DGI Loss: 0.6931 | Reg Loss: -0.2846 | Total: 0.5509
Epoch 6500 | DGI Loss: 0.6931 | Reg Loss: -0.2846 | Total: 0.5508
Epoch 7000 | DGI Loss: 0.6932 | Reg Loss: -0.2844 | Total: 0.5510
Epoch 7500 | D

In [65]:
model.eval()
with torch.no_grad():
    embeddings = model.get_embeddings(node_feats.to(device), edge_index.to(device))
    class_probabilities = F.softmax(model.classifier(embeddings), dim=1).cpu().numpy()

y_pred = np.argmax(class_probabilities, axis=1)

In [67]:
acc_score = accuracy_score(y, y_pred)
acc_score_inverted = accuracy_score(y, 1 - y_pred)
prec_score = precision_score(y, y_pred)
rec_score = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
log_loss_value = log_loss(y, class_probabilities)

print("Accuracy:", acc_score)
print("Accuracy (inverted):", acc_score_inverted)
print("Precision:", prec_score)
print("Recall:", rec_score)
print("F1:", f1)
print("Log Loss:", log_loss_value)

Accuracy: 0.678082191780822
Accuracy (inverted): 0.3219178082191781
Precision: 0.6976744186046512
Recall: 0.9183673469387755
F1: 0.7929515418502202
Log Loss: 0.6231503046531645


In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

hidden_dim   = 512
cut          = 0
dropout      = 0.3
num_runs     = 10
num_epochs   = 8000
lambda_list  = [0.1, 0.3, 0.5, 0.9]
base_seed    = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

node_feats = node_feats.to(device)
edge_index = edge_index.to(device)
A = A.to(device)

if isinstance(y, torch.Tensor):
    y_np = y.detach().cpu().numpy().astype(int)
else:
    y_np = np.asarray(y).astype(int)

N, feats_dim = node_feats.size(0), node_feats.size(1)

all_results = []
bce_loss = nn.BCEWithLogitsLoss()

for lam in lambda_list:
    print(f"\n================ LAMBDA = {lam} ================\n")

    acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

    for run in range(num_runs):
        print(f"\n--- Run {run+1}/{num_runs} ---")

        seed = base_seed + run
        torch.manual_seed(seed)
        np.random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)


        model = DGI_with_classifier(feats_dim, hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00001)
        scheduler = StepLR(optimizer, step_size=200, gamma=0.5)


        for epoch in range(num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            perm = torch.randperm(N, device=device)
            corrupt_features = node_feats[perm]

            logits, embeddings = model(node_feats, corrupt_features, edge_index)

            lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)])
            dgi_loss = bce_loss(logits.squeeze(), lbl)
            reg_loss = model.Reg_loss(A, embeddings)

            loss = dgi_loss + lam * reg_loss

            if epoch % 500 == 0:
                print(f"Epoch {epoch:4d} | DGI: {dgi_loss.item():.4f} | Reg: {reg_loss.item():.4f} | "
                      f"λ*Reg: {(lam * reg_loss).item():.4f} | Total: {loss.item():.4f}")

            loss.backward()
            optimizer.step()
            scheduler.step()

        model.eval()
        with torch.no_grad():
            emb = model.get_embeddings(node_feats, edge_index)
            logits_cls = model.classifier(emb)                   # [N, 2]
            class_probabilities = F.softmax(logits_cls, dim=1).cpu().numpy()
            y_pred = np.argmax(class_probabilities, axis=1)

        acc  = accuracy_score(y_np, y_pred)
        acc_inv = accuracy_score(y_np, 1 - y_pred)

        if acc_inv > acc:
            acc = acc_inv
            y_pred = 1 - y_pred
            class_probabilities = class_probabilities[:, ::-1]

        prec = precision_score(y_np, y_pred, zero_division=0)
        rec  = recall_score(y_np, y_pred, zero_division=0)
        f1   = f1_score(y_np, y_pred, zero_division=0)
        ll   = log_loss(y_np, class_probabilities)

        print(f"Run {run+1} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | LogLoss: {ll:.4f}")

        acc_scores.append(acc)
        prec_scores.append(prec)
        rec_scores.append(rec)
        f1_scores.append(f1)
        log_losses.append(ll)

    lambda_results = {
        "lambda": lam,
        "accuracy":  (float(np.mean(acc_scores)), float(np.std(acc_scores))),
        "precision": (float(np.mean(prec_scores)), float(np.std(prec_scores))),
        "recall":    (float(np.mean(rec_scores)), float(np.std(rec_scores))),
        "f1":        (float(np.mean(f1_scores)),  float(np.std(f1_scores))),
        "log_loss":  (float(np.mean(log_losses)), float(np.std(log_losses))),
    }
    all_results.append(lambda_results)

    print(f"\n--- RESULTS FOR LAMBDA = {lam} ---")
    print(f"Accuracy : {lambda_results['accuracy'][0]:.4f} ± {lambda_results['accuracy'][1]:.4f}")
    print(f"Precision: {lambda_results['precision'][0]:.4f} ± {lambda_results['precision'][1]:.4f}")
    print(f"Recall   : {lambda_results['recall'][0]:.4f} ± {lambda_results['recall'][1]:.4f}")
    print(f"F1 Score : {lambda_results['f1'][0]:.4f} ± {lambda_results['f1'][1]:.4f}")
    print(f"Log Loss : {lambda_results['log_loss'][0]:.4f} ± {lambda_results['log_loss'][1]:.4f}")

print("\n================ FINAL SUMMARY FOR ALL LAMBDAS ================\n")
print(f"{'Lambda':>8} | {'Accuracy':>18} | {'Precision':>18} | {'Recall':>18} | {'F1 Score':>18} | {'Log Loss':>18}")
print("-" * 108)
for res in all_results:
    print(f"{res['lambda']:>8} | "
          f"{res['accuracy'][0]:.4f} ± {res['accuracy'][1]:.4f} | "
          f"{res['precision'][0]:.4f} ± {res['precision'][1]:.4f} | "
          f"{res['recall'][0]:.4f} ± {res['recall'][1]:.4f} | "
          f"{res['f1'][0]:.4f} ± {res['f1'][1]:.4f} | "
          f"{res['log_loss'][0]:.4f} ± {res['log_loss'][1]:.4f}")


================ LAMBDA = 0.1 ================


--- Run 1/10 ---
Epoch    0 | DGI: 0.7094 | Reg: -0.2841 | λ*Reg: -0.0284 | Total: 0.6810
Epoch  500 | DGI: 0.6932 | Reg: -0.2841 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 1000 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 1500 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 2000 | DGI: 0.6931 | Reg: -0.2841 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 2500 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 3000 | DGI: 0.6931 | Reg: -0.2841 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 3500 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 4000 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 4500 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 5000 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 5500 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 | Total: 0.6647
Epoch 6000 | DGI: 0.6931 | Reg: -0.2842 | λ*Reg: -0.0284 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

hidden_dim   = 512
cut          = 0
dropout      = 0.25
num_runs     = 10
num_epochs   = 8000
lambda_list  = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]
base_seed    = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

node_feats = node_feats.to(device)
edge_index = edge_index.to(device)
A = A.to(device)

if isinstance(y, torch.Tensor):
    y_np = y.detach().cpu().numpy().astype(int)
else:
    y_np = np.asarray(y).astype(int)

N, feats_dim = node_feats.size(0), node_feats.size(1)

all_results = []
bce_loss = nn.BCEWithLogitsLoss()

for lam in lambda_list:
    print(f"\n================ LAMBDA = {lam} ================\n")

    acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

    for run in range(num_runs):
        print(f"\n--- Run {run+1}/{num_runs} ---")

        seed = base_seed + run
        torch.manual_seed(seed)
        np.random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)


        model = DGI_with_classifier(feats_dim, hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
        scheduler = StepLR(optimizer, step_size=200, gamma=0.5)


        for epoch in range(num_epochs + 1):
            model.train()
            optimizer.zero_grad()

            perm = torch.randperm(N, device=device)
            corrupt_features = node_feats[perm]

            logits, embeddings = model(node_feats, corrupt_features, edge_index)

            lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)])
            dgi_loss = bce_loss(logits.squeeze(), lbl)
            reg_loss = model.Reg_loss(A, embeddings)

            loss = dgi_loss + lam * reg_loss

            if epoch % 500 == 0:
                print(f"Epoch {epoch:4d} | DGI: {dgi_loss.item():.4f} | Reg: {reg_loss.item():.4f} | "
                      f"λ*Reg: {(lam * reg_loss).item():.4f} | Total: {loss.item():.4f}")

            loss.backward()
            optimizer.step()
            scheduler.step()

        model.eval()
        with torch.no_grad():
            emb = model.get_embeddings(node_feats, edge_index)
            logits_cls = model.classifier(emb)                   # [N, 2]
            class_probabilities = F.softmax(logits_cls, dim=1).cpu().numpy()
            y_pred = np.argmax(class_probabilities, axis=1)

        acc  = accuracy_score(y_np, y_pred)
        acc_inv = accuracy_score(y_np, 1 - y_pred)

        if acc_inv > acc:
            acc = acc_inv
            y_pred = 1 - y_pred
            class_probabilities = class_probabilities[:, ::-1]

        prec = precision_score(y_np, y_pred, zero_division=0)
        rec  = recall_score(y_np, y_pred, zero_division=0)
        f1   = f1_score(y_np, y_pred, zero_division=0)
        ll   = log_loss(y_np, class_probabilities)

        print(f"Run {run+1} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | LogLoss: {ll:.4f}")

        acc_scores.append(acc)
        prec_scores.append(prec)
        rec_scores.append(rec)
        f1_scores.append(f1)
        log_losses.append(ll)

    lambda_results = {
        "lambda": lam,
        "accuracy":  (float(np.mean(acc_scores)), float(np.std(acc_scores))),
        "precision": (float(np.mean(prec_scores)), float(np.std(prec_scores))),
        "recall":    (float(np.mean(rec_scores)), float(np.std(rec_scores))),
        "f1":        (float(np.mean(f1_scores)),  float(np.std(f1_scores))),
        "log_loss":  (float(np.mean(log_losses)), float(np.std(log_losses))),
    }
    all_results.append(lambda_results)

    print(f"\n--- RESULTS FOR LAMBDA = {lam} ---")
    print(f"Accuracy : {lambda_results['accuracy'][0]:.4f} ± {lambda_results['accuracy'][1]:.4f}")
    print(f"Precision: {lambda_results['precision'][0]:.4f} ± {lambda_results['precision'][1]:.4f}")
    print(f"Recall   : {lambda_results['recall'][0]:.4f} ± {lambda_results['recall'][1]:.4f}")
    print(f"F1 Score : {lambda_results['f1'][0]:.4f} ± {lambda_results['f1'][1]:.4f}")
    print(f"Log Loss : {lambda_results['log_loss'][0]:.4f} ± {lambda_results['log_loss'][1]:.4f}")

print("\n================ FINAL SUMMARY FOR ALL LAMBDAS ================\n")
print(f"{'Lambda':>8} | {'Accuracy':>18} | {'Precision':>18} | {'Recall':>18} | {'F1 Score':>18} | {'Log Loss':>18}")
print("-" * 108)
for res in all_results:
    print(f"{res['lambda']:>8} | "
          f"{res['accuracy'][0]:.4f} ± {res['accuracy'][1]:.4f} | "
          f"{res['precision'][0]:.4f} ± {res['precision'][1]:.4f} | "
          f"{res['recall'][0]:.4f} ± {res['recall'][1]:.4f} | "
          f"{res['f1'][0]:.4f} ± {res['f1'][1]:.4f} | "
          f"{res['log_loss'][0]:.4f} ± {res['log_loss'][1]:.4f}")


================ LAMBDA = 0.001 ================


--- Run 1/10 ---
Epoch    0 | DGI: 0.7107 | Reg: -0.2835 | λ*Reg: -0.0003 | Total: 0.7105
Epoch  500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 1000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 1500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 2000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 2500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 3000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 3500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 4000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 4500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 5000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 5500 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.0003 | Total: 0.6929
Epoch 6000 | DGI: 0.6931 | Reg: -0.2840 | λ*Reg: -0.000